In [1]:
1. 
# !pip install torch
# !pip install torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pandas as pd
from torchvision.io import read_image

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
4. 
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, names=['file_name', 'label'], skiprows=[0])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        try:
            image = read_image(img_path)
        except:
            print(self.img_labels.iloc[idx, 0])
            exit()
        label = int(self.img_labels.iloc[idx, 1])
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label




#Define Neural Networks Model.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 10)

    def forward(self, x):
        x = x.float()
        h1 = F.relu(self.fc1(x.view(-1, 784)))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        h4 = F.relu(self.fc4(h3))
        h5 = F.relu(self.fc5(h4))
        h6 = self.fc6(h5)
        return F.log_softmax(h6, dim=1)

#Prepare Data Loader for Training and Validation

transform = transforms.Compose([
                 transforms.ToTensor(),
                 transforms.Normalize((0.1307,), (0.3081,))])

print("init model done")



epochs = 10
lr = 0.01
momentum = 0.5
no_cuda = True
seed = 1
log_interval = 200

use_cuda = not no_cuda and torch.cuda.is_available()

torch.manual_seed(seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

print("set vars and device done")


batch_size = 64
test_batch_size = 1000

train_dataset = CustomImageDataset(
    annotations_file='./test_folder/FashionMNIST/train_annotations.csv',
    img_dir='./test_folder/FashionMNIST/train_iamges/',
    )

test_dataset = CustomImageDataset(
    annotations_file='./test_folder/FashionMNIST/test_annotations.csv',
    img_dir='./test_folder/FashionMNIST/test_iamges/',
    )


train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = 64, shuffle = True)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size = test_batch_size, shuffle = True, **kwargs)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)


def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(log_interval, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
          (test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

for epoch in range(1, 11):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(log_interval, model, device, test_loader)
torch.save(model, './model.pt')

init model done
set vars and device done
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.372018
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.469477
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.634011
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.411023
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.327723

Test set: Average loss: 0.4722, Accuracy: 8277/10000 (83%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.347718
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.237241
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.552695
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.439620
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.360786

Test set: Average loss: 0.4206, Accuracy: 8409/10000 (84%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.394550
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.462735
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.296683
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.210451
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.268203

Test set: Average loss: 0.3831, Accuracy: 8601/10000 (86%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.323

In [2]:
# 2. 
# daya_FashionMNIST_test, train ubyte download

training_data = datasets.FashionMNIST(
     root = "data",
     train = True,
     download = True,
     transform = ToTensor()
)

test_data = datasets.FashionMNIST(
     root = "data",
     train = False,
     download = True,
     transform = ToTensor()
)

In [4]:
# 3. test_folder 파일 내에 test_images(60,000), train_images(10,000), test_annotations.csv, train_annotations.csv 파일 print
# test_images(60,000), train_images(10,000) 양이 많아 임시 삭제 했습니다. 본 파일 실행 시 해당 부분 재 실행 후 파일 다운로드 이후 진행해야 합니다.
img_size = 28
num_images = 5

## save annotation csv
# header
train_test = [
    {
        'task_name': 'train',
        'image' : 'data/FashionMNIST/raw/train-images-idx3-ubyte',
        'label' : 'data/FashionMNIST/raw/train-labels-idx1-ubyte'
    },
    {
        'task_name': 'test',
        'image' : 'data/FashionMNIST/raw/t10k-images-idx3-ubyte',
        'label' : 'data/FashionMNIST/raw/t10k-labels-idx1-ubyte'
    }
]

for i in train_test : 
    imgf = open(i['image'],'rb')
    imgd = imgf.read(16)
    lblf = open(i['label'], 'rb')
    lbuf = lblf.read(8)

    df_dict = {
        'file_name' : [],
        'label' : []
    }

    idx = 0
    while True:
        imgd = imgf.read(img_size * img_size)
        if not imgd:
            break
        data = np.frombuffer(imgd, dtype=np.uint8).astype(float)
        data = data.reshape(1, img_size, img_size, 1)
        image = np.asarray(data).squeeze()
        lbld = lblf.read(1)
        labels = np.frombuffer(lbld, dtype=np.uint8).astype(np.int64)
        file_name = '{}.png'.format(idx)

        os.makedirs('./test_folder/FashionMNIST/{}_iamges'.format(i['task_name']), exist_ok = True)

        cv2.imwrite('./test_folder/FashionMNIST/{}_iamges/{}'.format(i['task_name'], file_name), image)

        idx +=1
        df_dict['label'].append(labels[0])
        df_dict['file_name'].append(file_name)

    # print(df_dict)
    import pandas as pd
    df = pd.DataFrame(df_dict)
    print(df)
    df.to_csv('./test_folder/FashionMNIST/{}_annotations.csv'.format(i['task_name']))

       file_name  label
0          0.png      9
1          1.png      0
2          2.png      0
3          3.png      3
4          4.png      0
...          ...    ...
59995  59995.png      5
59996  59996.png      1
59997  59997.png      3
59998  59998.png      0
59999  59999.png      5

[60000 rows x 2 columns]
     file_name  label
0        0.png      9
1        1.png      2
2        2.png      1
3        3.png      1
4        4.png      6
...        ...    ...
9995  9995.png      9
9996  9996.png      1
9997  9997.png      8
9998  9998.png      1
9999  9999.png      5

[10000 rows x 2 columns]
